In [1]:
import quepy
from SPARQLWrapper import SPARQLWrapper, JSON
import re
import json

In [2]:
sparql = SPARQLWrapper("http://argon-virtuoso:8890/sparql")
opendata = quepy.install("opendata")

example_questions = [
    "¿Dónde se ha citado Sanidad?",
    "¿Dónde trabaja Ángel Fernández?",
    "¿Qué puesto ocupa Ángel Fernández?",
    "¿Cuándo empezó a trabajar Manuel Muniesa Alfonso?",
    "¿Cuándo terminó de trabajar Manuel Muniesa Alfonso?",
    "¿Dónde está el Instituto Tecnológico de Aragón?",
    "¿Cuál es el teléfono del Instituto Tecnológico de Aragón?",
    "Resumen de http://transparencia.aragon.es/node/5528",
    "Categorías de http://transparencia.aragon.es/node/5528"
]

def quepy_query(question):
    question = question.replace("¿".decode("utf-8")," ¿ ".decode("utf-8")) # !TODO mejorar
    url = re.findall("https?://.*",question)
    if url:
        question = question.replace(url[0],"URL_MATCH") # Reemplazar la url por URL_MATCH

    targets, query, metadata = opendata.get_query(question)

    if not query:
        return {"message": "La pregunta no es válida, preguntas de ejemplo:", "results": example_questions, "answer": False}
    else:
        conceptTarget = metadata[0]
        infoToPrint = metadata[1]
        for i in range(len(targets)):
            if targets[i].startswith("?"):
                targets[i] = targets[i][1:]
        if url:
            query = query.replace("URL_MATCH", url[0])  # Deshacer el cambio de URL_MATCH
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        result = sparql.query().convert()
        ret = {}
        for r in result["results"]["bindings"]:
            # Obtener las 3 respuestas
            key = r[targets[2]]["value"]  # Nombre completo the la pregunta
            answer = r[targets[1]]["value"]  # Respuesta (literal)
            node = r[targets[0]]["value"]  # Nodo de la respuesta
            if key not in ret:
                ret[key] = [[answer,node]]
            else:
                ret[key].append([answer,node])
        if ret:
            if infoToPrint == "La lista de categorías es:":  # Limpiar la lista de categorías
                for key in ret:
                    for i in range(len(ret[key])):
                        ret[key][i][0] = ret[key][i][0].split("#")[1]
                    ret[key] = [x for x in ret[key] if x[0] != "NamedIndividual"]
            return {"message": infoToPrint, "results": ret, "answer": True}  # Respuesta encontrada
        else:
             return {"message": "No se ha encontrado una respuesta", "answer": False}  # Respuesta no encontrada en Virtuoso


In [3]:
REQUEST=json.dumps({
    'body': {'question':'¿Qué puesto ocupa Ángel Fernández?'
            }})
print(REQUEST)

{"body": {"question": "\u00bfQu\u00e9 puesto ocupa \u00c1ngel Fern\u00e1ndez?"}}


In [4]:
# POST /quepy
req = json.loads(REQUEST)
question = req['body']['question']
answer = quepy_query(question)
status = 200
print (json.dumps(answer))


{"answer": true, "message": "{0} trabaja como:", "results": {"\u00c1ngel Fern\u00e1ndez Cuello": [["Director del Instituto Tecnol\u00f3gico de Arag\u00f3n", "http://opendata.aragon.es/def/ei2a#Role739"]]}}


In [5]:
quepy_query("donde citar sanidad")

{'answer': False,
 'message': 'Respuesta no encontrada, busqueda alternativa en Solr:',
 'results': [u'https://www.saludinforma.es/portalsi/web/salud/detalle/-/asset_publisher/QIkE4rhakPHF/content/plan-de-salud-de-aragon-2030?redirect=https%3A%2F%2Fwww.saludinforma.es%2Fportalsi%2Fweb%2Fsalud%2Fdetalle%3Fp_p_id%3D101_INSTANCE_gcAH3p6QSwo7%26p_p_lifecycle%3D0%26p_p_state%3Dnormal%26p_p_mode%3Dview%26p_p_col_id%3Dcolumn-1%26p_p_col_count%3D2',
  u'http://transparencia.aragon.es/content/MesaSectorialSanidad',
  u'http://transparencia.aragon.es/node/5679',
  u'https://www.saludinforma.es/portalsi/documents/10179/2629916/ANEXO_VI_Aceptacion_subvencion_concedida.pdf/ba960b51-448d-4f71-b662-d7d5789e0920',
  u'https://www.saludinforma.es/portalsi/web/salud/proteccion-datos',
  u'http://transparencia.aragon.es/content/departamento-de-sanidad',
  u'https://www.saludinforma.es/portalsi/web/salud/servicios-prestaciones/carpeta-de-salud',
  u'https://www.saludinforma.es/portalsi/web/salud/servicios

In [6]:
# ResponseInfo POST /quepy
print(json.dumps({
    "status" : status,
    "headers" : {
        "Content-Type" : "application/json"
    }
}))

{"status": 200, "headers": {"Content-Type": "application/json"}}
